# Tutorial part 2

Here we'll continue learning about the ``PfLine`` class. 

In [part 1](part1.ipynb) we saw 'flat' portfolio lines, i.e., portfolio lines without nested children. These portfolio lines have a single value for each timestamp for each dimension (i.e., for a price-and-volume portfolio line: one price value, one power value, one energy value, one revenue value).

We'll look at nested portfolio lines in this part of the tutorial. 

## Example data

Let's first use the mock functions to get some timeseries and (flat) portfolio lines:

In [1]:
import portfolyo as pf
import pandas as pd

index = pd.date_range("2024", freq="D", periods=366)
# Creating some distinct timeseries (the details here are not important).
ts_offtake_B2C = -1 * pf.dev.w_offtake(index, avg=100, year_amp=0.3, week_amp=0.25)
ts_offtake_B2B = -1 * pf.dev.w_offtake(index, avg=50, year_amp=0.1, week_amp=0.05)
# turning these timeseries into portfolio lines.
offtake_B2C = pf.PfLine({"w": ts_offtake_B2C})
offtake_B2B = pf.PfLine({"w": ts_offtake_B2B})

We already saw that we can do arithmatic with portfolio lines, and by adding them together we can get the total offtake portfolio line:

In [2]:
offtake_B2C + offtake_B2B

PfLine object with volume information.
. Timestamps: first: 2024-01-01 00:00:00     timezone: None
               last: 2024-12-31 00:00:00         freq: <Day> (366 datapoints)
                                     w           q
                                    MW         MWh

2024-01-01 00:00:00             -176.2      -4 228
2024-01-02 00:00:00             -195.6      -4 694
2024-01-03 00:00:00             -196.3      -4 710
2024-01-04 00:00:00             -194.2      -4 660
2024-01-05 00:00:00             -196.5      -4 716
2024-01-06 00:00:00             -177.4      -4 258
2024-01-07 00:00:00             -143.2      -3 436
2024-01-08 00:00:00             -174.5      -4 189
2024-01-09 00:00:00             -198.4      -4 762
2024-01-10 00:00:00             -195.3      -4 688
..                                  ..          ..
2024-12-23 00:00:00             -174.3      -4 183
2024-12-24 00:00:00             -193.1      -4 635
2024-12-25 00:00:00             -192.8      -4 627
2024-1

This is again a flat, childless portfolio line; the contributions of the individual portfolios are lost. Often, this is exactly what we want.

## Portfolio line with children

In some circumstances, however, we might want to retain the individual data. This is possible: we can combine these two flat volume-only portfolio lines into a new volume-only portfolio line, by specifying in a dictionary that uses their names as keys. Like so:

In [3]:
offtake = pf.PfLine({"B2C": offtake_B2C, "B2B": offtake_B2B})
offtake

PfLine object with volume information.
. Timestamps: first: 2024-01-01 00:00:00     timezone: None
               last: 2024-12-31 00:00:00         freq: <Day> (366 datapoints)
. Children: 'B2C' (volume), 'B2B' (volume)
                                     w           q
                                    MW         MWh

2024-01-01 00:00:00             -176.2      -4 228
2024-01-02 00:00:00             -195.6      -4 694
2024-01-03 00:00:00             -196.3      -4 710
2024-01-04 00:00:00             -194.2      -4 660
2024-01-05 00:00:00             -196.5      -4 716
2024-01-06 00:00:00             -177.4      -4 258
2024-01-07 00:00:00             -143.2      -3 436
2024-01-08 00:00:00             -174.5      -4 189
2024-01-09 00:00:00             -198.4      -4 762
2024-01-10 00:00:00             -195.3      -4 688
..                                  ..          ..
2024-12-23 00:00:00             -174.3      -4 183
2024-12-24 00:00:00             -193.1      -4 635
2024-12-25 00:

Superficially, this is the exact same object as the one before it. And unless we specify that we are interested in one of the children, we are working with the exact same aggregate data.

### Accessing the children

However, as can be seen in the header, this portfolio line has children, which have the names we specified for them: ``"B2C"`` and ``"B2B"``. If we want, we can access these by using their names as an index, e.g.:

In [4]:
offtake["B2C"]

PfLine object with volume information.
. Timestamps: first: 2024-01-01 00:00:00     timezone: None
               last: 2024-12-31 00:00:00         freq: <Day> (366 datapoints)
                                     w           q
                                    MW         MWh

2024-01-01 00:00:00             -125.1      -3 002
2024-01-02 00:00:00             -144.0      -3 455
2024-01-03 00:00:00             -144.0      -3 457
2024-01-04 00:00:00             -143.3      -3 438
2024-01-05 00:00:00             -145.2      -3 484
2024-01-06 00:00:00             -127.2      -3 054
2024-01-07 00:00:00              -96.3      -2 312
2024-01-08 00:00:00             -124.9      -2 999
2024-01-09 00:00:00             -146.5      -3 516
2024-01-10 00:00:00             -144.1      -3 459
..                                  ..          ..
2024-12-23 00:00:00             -123.8      -2 971
2024-12-24 00:00:00             -141.2      -3 388
2024-12-25 00:00:00             -140.9      -3 382
2024-1

Nesting like this is not limited to one level.

## Arithmatic with children

All of the arithmatic operations that we can do with flat portfolio lines, we can also do with portfolio lines that have children.

In some operations, the children can be retained (i.e., are present in the resulting portfolio line). In some, however, they cannot, and the portfolio line is flattened. In that case, a ``PfLineFlattenedWarning`` is issued:

In [5]:
new_offtake = offtake - pf.Q_(10.0, "MWh")

c:\users\ruud.wijtvliet\ruud\python\dev\portfolyo\portfolyo\core\pfline\enable_arithmatic.py:82: PfLineFlattenedWarning: When adding a SinglePfLine and MultiPfLine, the MultiPfLine is flattened.
  warnings.warn(


See [the section on arithmatic](../core/pfline.rst#operations-arithmatic) for more details on when a portfolio line's children are retained and when not.

This tutorial is continued [in part 3](part3.ipynb).